In [ ]:
import numpy as np
from plots import *
import scipy.io
import pandas as pd
from pathlib import Path

def plot_mags(wf, title=None, t_win=1, sf=False):
    m = get_mags(wf, sr=44100, t_win=t_win, dict=True)
    mags = m['mags']
    freq_ax = m['freq_ax']
    plt.plot(freq_ax, np.log10(mags)*10)
    plt.title(title)
    if sf:
        plt.savefig(title + ".png")
    plt.show()
def plot_supp(path, sf=False):
    data = np.loadtxt(path)
    freqs = data[:, 0]
    mags = data[:, 1]
    plt.plot(freqs, mags)
    plt.title(str(path).split("\\")[-1])
    plt.show()

In [ ]:
df = pd.read_parquet('Curated Data.parquet')
for index, row in df.iterrows():
    wf = row['wf']
    filepath = row['filepath']
    if filepath.split("\\")[1] in ["Other", "Tree Cricket"]:
        continue 
    plot_mags(wf=wf, title=Path(filepath).stem, t_win=1, sf=True)